In [15]:
import pandas as pd
df = pd.read_csv('/content/item.csv',encoding='latin-1', on_bad_lines='skip')
item_names_column = 'name'
corpus = df[item_names_column].dropna().tolist()  # Drop NaN values and convert to list
print(corpus)

['Chana Samosa', 'Kashmiri Pulao', 'Chikku Milkshake', 'Chole Bhature', 'French Fries', 'Lachha Parotta', 'Onion Rava Dosai', 'Paneer Pulao', 'Paneer Kothu Idiyappam', 'Paneer Fried Rice', 'Crispy Applewood Smoked Bacon Sandwhich', 'Onion Dosai', 'Khumb Masala', 'Rava Kichadi', 'Paruppu Podi Dosa', 'Chicken Drumstick', 'Rasmalai', 'House Sliders', 'Dal Mixture', 'Aloo Matar', 'Curd Rice', 'Jeera Rice', 'Babycorn Chilli Fries', 'Pan Roasted Salmon Burger', 'Poondu Dosai (Garlic)', 'Roasted Turkey on Sourdough', 'Bombay Halwa', 'Peas Pulao', 'Strawberry Milkshake', 'Peri Peri Chicken Noodles', 'Kai Kari Kurma', 'Mexican Chicken Shawarma', 'Chinese Chicken Salad', "Ibrahim's Spl Bbq Paarai Fish", 'Egg Schezwan Rice', 'Masala Uthappam', 'South Indian Thali', 'Poondu Dosai (Garlic)', 'Chocolate', 'Four Cheese Ravioli', 'Ruffina Prosecco', 'Impossible Veg Kothu Parotta', 'Onion Dosa', 'Soda', 'Baby Corn Schezwan Fried Rice', 'Margherita Pizza', 'Bbq Boneless Chilli', 'Keerai Vadai', 'Rava Do

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Create input sequences using the tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and create predictors and label
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Build the model with improvements
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(total_words, activation='softmax'))

# Compile the model with a lower learning rate
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model with a larger number of epochs and possibly a different batch size
history = model.fit(X, y, epochs=150, batch_size=64, verbose=1)


Epoch 1/150
124/124 [==============================] - 29s 168ms/step - loss: 5.8938 - accuracy: 0.0342
Epoch 2/150
124/124 [==============================] - 22s 179ms/step - loss: 5.4222 - accuracy: 0.0500
Epoch 3/150
124/124 [==============================] - 21s 167ms/step - loss: 5.1448 - accuracy: 0.0615
Epoch 4/150
124/124 [==============================] - 22s 177ms/step - loss: 4.9267 - accuracy: 0.0796
Epoch 5/150
124/124 [==============================] - 22s 181ms/step - loss: 4.7604 - accuracy: 0.0955
Epoch 6/150
124/124 [==============================] - 21s 167ms/step - loss: 4.6199 - accuracy: 0.0988
Epoch 7/150
124/124 [==============================] - 22s 181ms/step - loss: 4.5248 - accuracy: 0.1084
Epoch 8/150
124/124 [==============================] - 21s 168ms/step - loss: 4.4076 - accuracy: 0.1275
Epoch 9/150
124/124 [==============================] - 22s 179ms/step - loss: 4.3028 - accuracy: 0.1441
Epoch 10/150
124/124 [==============================] - 23s 182m

In [ ]:
def predict_next_word(model, tokenizer, text):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None
# Test the prediction
seed_text = "Lachha"
next_word = predict_next_word(model, tokenizer, seed_text)
print(f"Seed text: {seed_text}")
print(f"Next word prediction: {next_word}")

In [ ]:
df=pd.read_csv("")